In [1]:
# if local use new_torch_env

import os

import numpy as np
import pandas as pd
# import seaborn as sns

from collections import Counter

import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from xml.etree import ElementTree, ElementInclude

import pickle

import detectron2

In [2]:
output_path = '/home/simon/Documents/Bodies/scripts/OD/Detectron2/faster_rcnn_X_101_32x8d_FPN_3x/outputs_list.pkl' 

with open(output_path, 'rb') as file:
    outputs_list = pickle.load(file)

In [3]:
from detectron2.utils.visualizer import Visualizer, ColorMode

In [4]:
from collections import Counter
import pandas as pd

In [5]:
# fake thing for sim:
letters = [chr(i) for i in range(ord('a'),ord('x'))]
numbers = range(0, len(letters))
feature_dict = dict(zip(numbers, letters))

img_id = 1
# ----------------------------------------


all_img_feature_list = [] # to create the slim df, its easier this way...
output_list_np = []

for instance in outputs_list: # so this is given each instance
    img_dict = {'img_id': img_id, 'scores': None , 'pred_classes': None}
    img_dict['scores'] = instance.scores.numpy()
    img_dict['pred_classes'] = instance.pred_classes.numpy()

    img_featureInt_count = dict(Counter(instance.pred_classes.numpy())) # the int encoding of the feature

    img_dict = {**img_dict, **img_featureInt_count}

    img_feature_list = [feature_dict[i] for i in instance.pred_classes.numpy()]
    img_feature_count = dict(Counter(img_feature_list)) # the actual feature name

    img_dict = {**img_dict, **img_feature_count}
    
    output_list_np.append(img_dict)
    all_img_feature_list += img_feature_list


# Outputs:
all_img_feature_list = list(set(all_img_feature_list)) # get the unique set of features

df_thick = pd.DataFrame(output_list_np).fillna(0)
df_thick.iloc[:,3:] = df_thick.iloc[:,3:].astype('int') # counts as ints not floats 

df_thin = df_thick[['img_id'] + all_img_feature_list].copy() # making a df with just feature counts and img_id

#return(all_img_feature_list, df_thick, df_thin)

In [6]:
df_thin

,img_id,h,c,a,s,r,d,q,m,l,f,b,n
0,1,0,0,3,0,3,0,0,0,3,0,0,3
1,1,0,0,2,0,3,0,0,1,2,0,0,3
2,1,0,1,1,0,0,0,0,0,2,0,0,2
3,1,0,0,4,1,0,3,0,0,7,1,0,7
4,1,1,0,1,0,3,0,0,0,2,0,0,3
5,1,0,0,2,0,0,0,1,0,1,1,0,2
6,1,0,0,0,0,0,0,0,0,0,0,1,0
7,1,0,1,1,0,2,0,0,1,2,0,0,1
8,1,0,0,1,0,1,0,0,0,3,0,0,3
9,1,0,0,4,0,1,2,4,0,5,3,0,6
